In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, LinearConstraint, Bounds
# import libpysal.weights as lpw

In [ ]:
# function that makes coordincates of data points
def make_coords(N_points, xmu, xsd, ymu, ysd):  
    x_coords = np.random.normal(xmu, xsd, size=N_points)
    y_coords = np.random.normal(ymu, ysd, size=N_points)
    coords = np.column_stack((x_coords, y_coords))
    return coords

In [ ]:
    # function that calculates distances between coordinates
    def distances(coords):
        N = len(coords)
        
        # Initialize an empty distances matrix
        distances = np.zeros((N, N))

        # Compute the euclidean distance between pair of coords
        for i in range(N):
            for j in range(i+1, N):
                distances[i,j] = distances[j,i] = np.linalg.norm(coords[i] - coords[j])       

        return distances

In [ ]:
np.random.seed(24)

p = 9
X = make_coords(p,0,1,0,1)
disX = distances(X)
paramTrue = [1,1]
SigmaTrue = paramTrue[0]*np.exp(-disX/paramTrue[1])
# np.linalg.eig(Sigma)[0].min()

# print(X)

In [ ]:

SigmaHalf = np.linalg.cholesky(SigmaTrue)
U = np.random.normal(0, 1, p).reshape(p, 1)
Y = np.dot(SigmaHalf, U)
# print(Y)


In [ ]:
def Sigma(param, dis):
  theta1, theta2 = param[0], param[1]
  cov = theta1*np.exp(-dis/theta2)
  return cov

In [ ]:
# define log likelihood function
def logLikelihood(param, Y):
    
    p = len(Y)
    sig = Sigma(param, disX)
    inv_Sigma = np.linalg.inv(sig)
    
    log_det_Sigma = 2 * np.sum(np.log(np.diag(np.linalg.cholesky(sig)))) # reduce computation complexity

    # det_Sigma = np.linalg.det(sig)
    # log_det_Sigma = np.log(det_Sigma)
        
    return 0.5 * Y.T @ inv_Sigma @ Y + 0.5 * log_det_Sigma


In [ ]:
param0 = paramTrue

# Define the constraint function
def constraint1(param, Y):
    p = len(Y)
    sig = Sigma(param, p)
    eigenvalues = np.linalg.eig(sig)[0]
    return eigenvalues.min() - 1e-10 # Constraints: eigenvalues > 0

                
# Define the optimization problem
problem = {
    'fun': logLikelihood,         # Objective function: minimize the minimum eigenvalue
    'x0': paramTrue,            # Initial guess for the parameters
    'args': (Y,),             # Additional arguments for the objective function 
    'constraints': {'type': 'eq', 'fun': constraint1, 'args':(Y,)},  # Constraint function: eigenvalues >= 0 ineq
    'method': 'SLSQP'        
}

result = minimize(**problem)


estimated_params = result.x

LinAlgError: ignored

In [ ]:
estimated_params